## POS Tagging với thư viện Hidden Markov Model và Classifier-Based Tagging

In [1]:
import nltk

Đọc file và chuẩn hóa dữ liệu để đưa vào train.

In [ ]:
with open('vi_train.pos', 'r') as train_doc:
  lines = train_doc.readlines()

  doc_for_train = []
  for line in lines:
    if line != "\n":
      doc_for_train.append([])
      word_tag = line.split()
      for w in word_tag:
        w = w.split("/")
        doc_for_train[-1].append(tuple(w))
doc_for_train[1]

Xác suất xuất hiện của các từ, cặp pos tagging, từng pos tagging riêng lẻ.

In [3]:
def count_probability(filename):
  word_hash = {}
  pos_hash = {}
  pos_counts = {}

  with open(filename, 'r') as train_doc:
    lines = list(train_doc.read().splitlines())
    
  n = len(lines)//2
  for i in range(n):
    if lines[i] == "":
      del lines[i]
  del lines[-1]

  pos_counts['S'] = 0
  for line in lines:
    prev_tag = 'S'
    pos_counts['S'] += 1
    word_tag = []
    word_tag = line.split(" ")
    for w in word_tag:
      temp = w.split("/")
      word = temp[0]
      pos = temp[-1]

      if (pos,word) in word_hash:
        word_hash[(pos, word)] += 1
      else:
        word_hash[(pos, word)] = 1
      
      if (prev_tag, pos) in pos_hash:
        pos_hash[(prev_tag, pos)] += 1
      else:
        pos_hash[(prev_tag, pos)] = 1

      if pos in pos_counts:
        pos_counts[pos] += 1
      else:
        pos_counts[pos] = 1

      prev_tag = pos

  for (pos, word) in word_hash.keys():
    word_hash[(pos, word)] = (word_hash[(pos, word)] / pos_counts[pos])
  for (pos1, pos2) in pos_hash.keys():
    pos_hash[(pos1, pos2)] = (pos_hash[(pos1, pos2)] / pos_counts[pos1])

  return (word_hash, pos_hash, pos_counts)

word_hash, pos_hash, pos_counts = count_probability('vi_train.pos')
print(word_hash)
print(pos_hash)
print(pos_counts)

{('E', 'Trên'): 0.002571973782460798, ('N', 'đường'): 0.007465618860510805, ('V', 'đi'): 0.02317863351419691, (',', ','): 0.9999049520007603, ('V', 'có'): 0.04430279723963546, ('M', 'một'): 0.26016597510373446, ('N', 'lần'): 0.006919886487666448, ('N', 'xe'): 0.00351451648111766, ('N', 'cô'): 0.0053481772538747, ('R', 'suýt'): 0.00021156558533145275, ('V', 'rơi'): 0.0007111626191855871, ('R', 'xuống'): 0.004090267983074753, ('N', 'vực'): 0.00010914647456887143, ('E', 'ở'): 0.07508504106861362, ('N', 'đèo'): 0.00015280506439642, ('Np', 'Ngoạn_Mục'): 0.0002630194634402946, ('.', '.'): 1.0, ('E', 'Trong'): 0.015265908902347964, ('N', 'trận'): 0.0025758567998253657, ('V', 'đánh'): 0.002238845282621293, ('A', 'ác_liệt'): 0.0007504377553572918, ('N', 'bên'): 0.0046932984064614715, ('N', 'thành'): 0.00030561012879284, ('N', 'cổ'): 0.0006548788474132286, ('Np', 'Quảng_Trị'): 0.0011835875854813256, ('N', 'loạt'): 0.0001309757694826457, ('N', 'đạn'): 0.0004365858982754857, ('N', 'pháo'): 0.00013

Train dữ liệu 'vi_train.pos' bằng Hidden Markov Model

In [7]:
from nltk.tag import hmm
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train(doc_for_train)

Tải lên dữ liêu 'vi_test.pos' và chuẩn hóa.

In [8]:
document = []
tag = []

with open('vi_test.pos') as test_doc:
  lines = test_doc.readlines()

for line in lines:
  if line != "\n":
    word_tag = line.split()
    sent = ""
    for w in word_tag:
      temp = w.split("/")
      w_tag = temp[-1]
      w_letter = temp[0]

      sent += w_letter + " "
      tag.append(w_tag)
    document.append(sent)
print (tag)
print (document)

['E', 'N', 'V', 'A', 'N', 'N', '-', 'N', ',', 'N', '-', 'N', ',', 'C', 'X', 'C', 'T', 'N', ',', 'X', 'R', 'V', 'R', 'V', 'N', '.', 'E', 'N', 'Ny', 'A', ',', 'N', 'V', 'N', 'V', 'R', 'V', 'C', 'V', ',', 'A', 'Np', '.', '.', '.', 'V', 'R', ',', 'N', 'V', 'V', 'V', '.', 'V', ',', 'V', 'N', ',', 'V', 'N', ',', 'V', 'A', ',', 'V', 'N', ',', 'A', '.', '.', '.', 'C', 'N', 'E', 'L', 'N', ',', 'L', 'N', 'V', 'E', 'L', 'N', 'A', 'N', 'P', 'V', 'R', 'A', 'X', ',', 'R', 'T', 'V', 'N', 'V', '.', 'E', 'L', 'N', 'P', ',', 'N', 'N', 'V', 'V', 'C', '"', 'N', '"', 'R', 'A', 'C', 'Nc', 'V', 'R', 'A', '.', 'N', 'N', 'V', 'E', 'N', ',', 'V', 'V', 'E', 'N', 'E', 'V', 'Np', 'R', 'V', 'M', 'Nu', ',', 'V', 'M', 'Nu', 'V', 'E', 'N', 'V', 'R', 'N', '.', 'Nc', 'Np', 'P', 'V', 'X', 'E', 'Nc', 'N', 'N', 'E', 'N', 'N', 'V', 'R', '.', 'N', 'V', ',', 'N', 'V', 'R', 'V', 'M', 'N', 'V', 'R', '.', '.', '.', 'Nc', 'N', '.', 'V', 'V', 'C', 'R', 'V', ',', 'R', 'A', '.', 'V', 'E', 'N', 'P', 'V', 'N', 'N', 'N', 'M', ',', 'N',

Xây dựng lại câu và đưa vào tập test

In [9]:
def tagging_split(tag_sentence):
  pos_tagging = []
  w = []
  tags = []
  for w_tag in tag_sentence:
    w.append(w_tag[0])
    tags.append(w_tag[-1])

  pos_tagging.append(w)
  pos_tagging.append(tags)
  return pos_tagging

predict_doc = []
for sent in document:
  predict = tagger.tag(sent.split())
  pos_tagging = tagging_split(predict)
  predict_doc.extend(pos_tagging[-1])



Kiểm tra độ dài tập test

In [10]:
print(len(predict_doc))
print(len(tag))

23542
23542


Tính tỉ lệ test đúng trên tập dữ liệu

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(predict_doc, tag)

0.6154107552459435

Tỉ lệ đúng ~ 62%

Dự đoán câu đầu vào: "Dù khá đắt nhưng tôi vẫn đồng ý ."

In [12]:
str_input = "Dù khá đắt nhưng tôi vẫn đồng_ý ."
result = tagger.tag(str_input.split())

print(result)

[('Dù', 'C'), ('khá', 'R'), ('đắt', 'A'), ('nhưng', 'C'), ('tôi', 'P'), ('vẫn', 'R'), ('đồng_ý', 'V'), ('.', '.')]


Chuẩn hóa dữ liệu test thành các cặp "từ"-"nhãn" (word-tag)

In [14]:
with open('vi_test.pos') as test_doc:  
  lines = test_doc.readlines()

doc_for_test = []
for line in lines:
  if line != "\n":
    doc_for_test.append([])
    word_tag = line.split()
    for w in word_tag:
      w = w.split("/")
      doc_for_test[-1].append(tuple(w))
# doc_for_test[1]

Sử dụng model Classifier Based POS Tagger để train và test dữ liệu

In [15]:
from nltk.tag.sequential import ClassifierBasedPOSTagger  
  
# initializing training and testing set     
train_data = doc_for_train
test_data = doc_for_test
  
tagging = ClassifierBasedPOSTagger(train = train_data) 
  
a = tagging.evaluate(test_data) 
  
print ("Accuracy : ", a)

Accuracy :  0.887859994902727


Kết quả train tốt hơn: ~89%

Sử dụng mô hình để dự đoán câu đầu vào: "Dù khá đắt nhưng tôi vẫn đồng_ý ."

In [16]:
# print(test_data[1])
str_test = "Dù khá đắt nhưng tôi vẫn đồng_ý ."
tagging.tag(str_test.split())

[('Dù', 'C'),
 ('khá', 'R'),
 ('đắt', 'A'),
 ('nhưng', 'C'),
 ('tôi', 'P'),
 ('vẫn', 'R'),
 ('đồng_ý', 'V'),
 ('.', '.')]